## Text Classification using NLP

Here in this project, I will try to train a machine learning model capable of predicting whether a given movie review is positive or negative. This is an example of 'sentiment analysis' where people's sentiments towards a particular entity are classified into different categories.

The dataset that I am going to use for this work consists of a total of 2000 documents. Half of the documents contain positive reviews regarding a movie while the remaining half contains negative reviews.

### Importing Libraries

In [86]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\T.K.DAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Importing the Dataset

In [87]:
movie_data = load_files(r'C:\Text Classification data\txt_sentoken')
X, y = movie_data.data, movie_data.target

The load_files will treat each folder inside the "txt_sentoken" folder as one category and all the documents inside that folder will be assigned its corresponding category. The load_files function loads the data from both "neg" and "pos" folders into the X variable, while the target categories are stored in y. Here X is a list of 2000 string type elements where each element corresponds to single user review. Similarly, y is a numpy array of size 2000. 

### Text Preprocessing

In [88]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\T.K.DAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [89]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

Here I have used Regex Expressions from Python re library to perform different preprocessing tasks. All non-word characters such as special characters, numbers, etc. have been removed initially.

Next, I have removed all the single characters. To remove such single characters, \s+[a-zA-Z]\s+ regular expression  has been  used which substitutes all the single characters having spaces on either side, with a single space.

Next, \^[a-zA-Z]\s+ regular expression has been used to replace a single character from the beginning of the document, with a single space. Replacing single characters with a single space may result in multiple spaces, which is not ideal. So I have used the regular expression \s+ to replace one or more spaces with a single space. 

In lemmatization, the word has been reduced into dictionary root form. For instance "cats" is converted into "cat"

### Converting Text to Numbers

In [90]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

Here I have used the bag of words model to convert text documents into corresponding numerical features.

### Finding TFIDF

The bag of words approach works fine for converting text to numbers. However, it has one drawback. It assigns a score to a word based on its occurrence in a particular document. It doesn't take into account the fact that the word might also be having a high frequency of occurrence in other documents as well. TFIDF resolves this problem.

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

### Train - Test Split

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 20% test set and 80% training set

### Training Text Classification Model and Predicting Sentiment

In [93]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=1000, random_state=0)

In [94]:
y_pred = classifier.predict(X_test)

### Evaluating the Model

To evaluate the performance of a classification model that I have just trained, I have used metrics such as the confusion matrix, F1 measure, and the accuracy.

In [95]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[180  28]
 [ 30 162]]
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       208
           1       0.85      0.84      0.85       192

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

0.855


#### From the output, it can be seen that the  model achieved an accuracy of 85.5%, which is quite good given the fact that I have randomly chosen all the parameters for CountVectorizer as well as for our random forest algorithm.